# **BLS Data Aggregation and Merge for Loiusiana Labor Economics Dashboard**  
**Version 1.01**  
Last updated: 4-11-2025

---

## Written by:  
### **Asif Rasool, Ph.D.**  

Research Economist  
Business Research Center  
Southeastern Louisiana University  
1514 Martens Drive, Hammond, LA 70401  
Phone: 985-549-3831  
Email: [asif.rasool@southeastern.edu](mailto:asif.rasool@southeastern.edu)  

---

## Table of Contents
1. [Introduction](#Introduction)
2. [Load API Key](#Load-API-Key)
3. [Fetch National-Level Data](#Fetch-National-Level-Data)
4. [Define LA Parish FIPS Codes](#Define-LA-Parish-FIPS-Codes)
5. [Build County-Level Series IDs](#Build-County-Level-Series-IDs)
6. [Fetch County-Level Data](#Fetch-County-Level-Data)
7. [Reshape County-Level Data](#Reshape-County-Level-Data)
8. [Merge Datasets](#Merge-Datasets)
9. [Main Execution](#Main-Execution)
10. [Conclusion](#Conclusion)

## Introduction

This notebook downloads national-level and Louisiana parish-level data from the BLS API across two time segments (1990-2009 and 2010-2025). 
The data for four key metrics (labor force, employment, unemployment, and unemployment rate) are fetched, reshaped, and merged such that each county-level record (by parish and date) gets its corresponding national-level data. 

Finally, the merged dataset is saved to a CSV file.

In [ ]:
import json
import time
from datetime import datetime
from pathlib import Path

import pandas as pd
import requests
import toml


## Load API Key

The following function loads the BLS API key from the secure TOML file. Make sure the file path is correct.

In [ ]:
def load_bls_api_key() -> str:
    """
    Load the BLS API key from the secrets TOML file.
    
    Returns:
        str: The API key.
    """
    config_path = Path(r"C:\Users\w0800598\Documents\Data-Science-Projects\Economics Dash\.streamlit\secrets.toml")
    config = toml.load(config_path)
    return config["BLS"]["api_key"]

# Example usage:
print("BLS API Key Loaded:", load_bls_api_key()[:4] + "...")

## Fetch National-Level Data

This function retrieves national-level BLS data for the specified period and merges the data for all four metrics based on the date.

In [ ]:
def get_national_data(start_year: str, end_year: str) -> pd.DataFrame:
    """
    Fetch national-level data from the BLS API for a given time segment.
    
    Args:
        start_year (str): The starting year.
        end_year (str): The ending year.
    
    Returns:
        pd.DataFrame: DataFrame containing national data with 'date' and metric columns.
    """
    api_key = load_bls_api_key()
    series_ids = {
        "labor_force": "LNS11000000",
        "employment": "LNS12000000",
        "unemployment": "LNS13000000",
        "unemployment_rate": "LNS14000000",
    }
    headers = {"Content-type": "application/json"}
    payload = {
        "seriesid": list(series_ids.values()),
        "startyear": start_year,
        "endyear": end_year,
        "registrationKey": api_key,
    }
    data = json.dumps(payload)
    url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
    response = requests.post(url, data=data, headers=headers)
    json_data = response.json()
    if "Results" not in json_data:
        print("Error fetching national data:", json.dumps(json_data, indent=2))
        return pd.DataFrame()

    dfs = {}
    for series in json_data["Results"]["series"]:
        sid = series["seriesID"]
        metric = next((k for k, v in series_ids.items() if v == sid), None)
        if not metric:
            continue
        df = pd.DataFrame(series["data"])
        df = df[df["period"].str.startswith("M")]
        df["year"] = df["year"].astype(str)
        df["month"] = df["period"].str[1:].str.zfill(2)
        df["date"] = pd.to_datetime(df["year"] + "-" + df["month"] + "-01")
        df["value"] = pd.to_numeric(df["value"], errors="coerce")
        df = df[["date", "value"]].rename(columns={"value": metric})
        dfs[metric] = df

    national_df = None
    for df in dfs.values():
        if national_df is None:
            national_df = df
        else:
            national_df = pd.merge(national_df, df, on="date", how="inner")
    if national_df is not None:
        national_df = national_df.sort_values("date")
    return national_df

# Example: Fetch national data for 1990-2009
national_data_1 = get_national_data("1990", "2009")
print("National data (1990-2009):", national_data_1.shape)

## Define LA Parish FIPS Codes

The function below returns a dictionary mapping Louisiana parish names to their corresponding three-digit county FIPS codes.

In [ ]:
def get_la_county_fips() -> dict:
    """
    Return a dictionary mapping Louisiana parish names to their county FIPS codes.
    Returns:
        dict: Parish names as keys and three-digit county FIPS codes as values.
    """
    return {
        "Acadia Parish": "001",
        "Allen Parish": "003",
        "Ascension Parish": "005",
        "Assumption Parish": "007",
        "Avoyelles Parish": "009",
        "Beauregard Parish": "011",
        "Bienville Parish": "013",
        "Bossier Parish": "015",
        "Caddo Parish": "017",
        "Calcasieu Parish": "019",
        "Caldwell Parish": "021",
        "Cameron Parish": "023",
        "Catahoula Parish": "025",
        "Claiborne Parish": "027",
        "Concordia Parish": "029",
        "De Soto Parish": "031",
        "East Baton Rouge Parish": "033",
        "East Carroll Parish": "035",
        "East Feliciana Parish": "037",
        "Evangeline Parish": "039",
        "Franklin Parish": "041",
        "Grant Parish": "043",
        "Iberia Parish": "045",
        "Iberville Parish": "047",
        "Jackson Parish": "049",
        "Jefferson Parish": "051",
        "Jefferson Davis Parish": "053",
        "Lafayette Parish": "055",
        "Lafourche Parish": "057",
        "La Salle Parish": "059",
        "Lincoln Parish": "061",
        "Livingston Parish": "063",
        "Madison Parish": "065",
        "Morehouse Parish": "067",
        "Natchitoches Parish": "069",
        "Orleans Parish": "071",
        "Ouachita Parish": "073",
        "Plaquemines Parish": "075",
        "Pointe Coupee Parish": "077",
        "Rapides Parish": "079",
        "Red River Parish": "081",
        "Richland Parish": "083",
        "Sabine Parish": "085",
        "St. Bernard Parish": "087",
        "St. Charles Parish": "089",
        "St. Helena Parish": "091",
        "St. James Parish": "093",
        "St. John the Baptist Parish": "095",
        "St. Landry Parish": "097",
        "St. Martin Parish": "099",
        "St. Mary Parish": "101",
        "St. Tammany Parish": "103",
        "Tangipahoa Parish": "105",
        "Tensas Parish": "107",
        "Terrebonne Parish": "109",
        "Union Parish": "111",
        "Vermilion Parish": "113",
        "Vernon Parish": "115",
        "Washington Parish": "117",
        "Webster Parish": "119",
        "West Baton Rouge Parish": "121",
        "West Carroll Parish": "123",
        "West Feliciana Parish": "125",
        "Winn Parish": "127",
    }


## Build County-Level Series IDs

We create full series IDs for all four metrics by combining the state FIPS (`22`), the county FIPS, a constant string (`000000`), and the metric code.

In [ ]:
def build_all_county_series_ids() -> dict:
    """
    Build a dictionary mapping full series IDs for all four metrics to parish and metric info.
    
    Returns:
        dict: Mapping of series ID to a dict with parish and metric.
    """
    fips = get_la_county_fips()
    metrics = {
        "labor_force": "0006",
        "employment": "0005",
        "unemployment": "0004",
        "unemployment_rate": "0003",
    }
    series_map = {}
    for parish, county_fips in fips.items():
        for metric, code in metrics.items():
            series_id = f"LAUCN22{county_fips}000000{code}"
            series_map[series_id] = {"parish": parish, "metric": metric}
    return series_map


## Fetch County-Level Data

This function fetches county-level data (for all four metrics) from the BLS API for a specified time segment (e.g. 1990-2009 or 2010-2025).

In [ ]:
def fetch_all_county_data(api_key: str, series_map: dict, start_year: str, end_year: str) -> pd.DataFrame:
    """
    Fetch county-level data from the BLS API for a given time segment.
    
    Args:
        api_key (str): The BLS API key.
        series_map (dict): Mapping of series IDs to parish and metric info.
        start_year (str): The starting year.
        end_year (str): The ending year.
    
    Returns:
        pd.DataFrame: Raw county-level data.
    """
    all_data = []
    url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
    headers = {"Content-type": "application/json"}
    series_ids = list(series_map.keys())

    for i in range(0, len(series_ids), 50):
        batch_ids = series_ids[i:i + 50]
        payload = {
            "seriesid": batch_ids,
            "startyear": start_year,
            "endyear": end_year,
            "registrationKey": api_key,
        }
        data = json.dumps(payload)
        response = requests.post(url, data=data, headers=headers)
        result = response.json()
        if "Results" not in result:
            print(f"⚠️ Error in batch {i}: {json.dumps(result, indent=2)}")
            continue
        for series in result["Results"]["series"]:
            sid = series["seriesID"]
            meta = series_map[sid]
            parish = meta["parish"]
            metric = meta["metric"]
            for entry in series["data"]:
                if entry["period"].startswith("M") and entry["value"] != "-":
                    month = entry["period"][1:].zfill(2)
                    try:
                        date = pd.to_datetime(f"{entry['year']}-{month}-01")
                    except Exception as e:
                        print(f"Error parsing date for {sid}: {e}")
                        continue
                    try:
                        value = float(entry["value"])
                    except ValueError:
                        continue
                    all_data.append({
                        "parish": parish,
                        "date": date,
                        "metric": metric,
                        "value": value,
                    })
        time.sleep(1)  # Respect API rate limits
    return pd.DataFrame(all_data)


## Reshape County-Level Data

This cell pivots the raw county-level data so that each row (unique parish and date) includes columns for all metrics.

In [ ]:
def reshape_county_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Pivot the county-level data so that each row (unique parish and date) has columns for all four metrics.
    
    Args:
        df (pd.DataFrame): Raw county-level data.
    
    Returns:
        pd.DataFrame: Pivoted DataFrame.
    """
    if df.empty:
        return df
    pivot_df = df.pivot_table(index=["parish", "date"], columns="metric", values="value").reset_index()
    desired_order = ["parish", "date", "labor_force", "employment", "unemployment", "unemployment_rate"]
    pivot_df = pivot_df.reindex(columns=desired_order)
    return pivot_df.sort_values(["parish", "date"])


## Merge Datasets

We now merge the national-level and county-level datasets. The national metric columns are renamed with a `national_` prefix before merging on the `date` column.

In [ ]:
def merge_datasets(national_df: pd.DataFrame, county_df: pd.DataFrame) -> pd.DataFrame:
    """
    Merge the national and county datasets on the 'date' column.
    National metric columns are renamed with a 'national_' prefix before merging.
    
    Args:
        national_df (pd.DataFrame): National-level data.
        county_df (pd.DataFrame): County-level data.
    
    Returns:
        pd.DataFrame: Merged DataFrame.
    """
    national_df = national_df.rename(columns={
        "labor_force": "national_labor_force",
        "employment": "national_employment",
        "unemployment": "national_unemployment",
        "unemployment_rate": "national_unemployment_rate",
    })
    merged = pd.merge(county_df, national_df, on="date", how="left")
    return merged.sort_values(["parish", "date"])


## Main Execution

We fetch national-level and county-level data for two segments (1990-2009 and 2010-2025), merge them, and save the final result.

In [ ]:
def main() -> None:
    # Fetch national-level data in two segments
    print("Fetching national-level data (1990-2009)...")
    national_df_1 = get_national_data("1990", "2009")
    print("Fetching national-level data (2010-2025)...")
    national_df_2 = get_national_data("2010", "2025")
    national_df = pd.concat([national_df_1, national_df_2]).sort_values("date")
    print("National data range:", national_df["date"].min(), "to", national_df["date"].max())

    # Fetch county-level data in two segments
    print("Fetching county-level data (1990-2009)...")
    api_key = load_bls_api_key()
    series_map = build_all_county_series_ids()
    raw_county_df_1 = fetch_all_county_data(api_key, series_map, "1990", "2009")
    print("Fetching county-level data (2010-2025)...")
    raw_county_df_2 = fetch_all_county_data(api_key, series_map, "2010", "2025")
    raw_county_df = pd.concat([raw_county_df_1, raw_county_df_2])
    county_df = reshape_county_data(raw_county_df)

    if not county_df.empty:
        print("County-level data sample:")
        print(county_df.head())
        print("County data range by parish:")
        print(county_df.groupby("parish")["date"].agg(["min", "max"]))
    else:
        print("No county-level data returned.")

    # Merge the national and county data
    merged_df = merge_datasets(national_df, county_df)
    print("Merged dataset sample:")
    print(merged_df.head())

    # Save the final merged dataset to a CSV file
    merged_df.to_csv("merged_national_county_1990_2025.csv", index=False)
    print("✅ Saved merged dataset to 'merged_national_county_1990_2025.csv'")


if __name__ == "__main__":
    main()


## Conclusion

This notebook downloads, processes, and merges national and county-level BLS data from two segments (1990-2009 and 2010-2025) and saves the final merged dataset to a CSV file.